## MNIST Digit Recognition

### Using lasagne NeuralNet

In [1]:
import sys
import os
import time
import numpy as np

#import theano
#import theano.tensor as T
#import lasagne

In [2]:
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
from lasagne.nonlinearities import softmax

C:\Users\babsab\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Load Data

In [3]:
def load_dataset():
    # We first define a download function, supporting both Python 2 and 3.
    if sys.version_info[0] == 2:
        from urllib import urlretrieve
    else:
        from urllib.request import urlretrieve
    
    def download(filename, source='http://yann.lecun.com/exdb/mnist/'):
        print("Downloading %s" % filename)
        urlretrieve(source + filename, filename)
    # We then define functions for loading MNIST images and labels.
    # For convenience, they also download the requested files if needed.
    import gzip
    
    def load_mnist_images(filename):
        if not os.path.exists(filename):
            download(filename)
        # Read the inputs in Yann LeCun's binary format.
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
        # The inputs are vectors now, we reshape them to monochrome 2D images,
        # following the shape convention: (examples, channels, rows, columns)
        data = data.reshape(-1, 1, 28, 28)
        # The inputs come as bytes, we convert them to float32 in range [0,1].
        # (Actually to range [0, 255/256], for compatibility to the version
        # provided at http://deeplearning.net/data/mnist/mnist.pkl.gz.)
        return data / np.float32(256)
    
    def load_mnist_labels(filename):
        if not os.path.exists(filename):
            download(filename)
        # Read the labels in Yann LeCun's binary format.
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=8)
        # The labels are vectors of integers now, that's exactly what we want.
        return data
    
    # We can now download and read the training and test set images and labels.
    X_train = load_mnist_images('train-images-idx3-ubyte.gz')
    y_train = load_mnist_labels('train-labels-idx1-ubyte.gz')
    X_test = load_mnist_images('t10k-images-idx3-ubyte.gz')
    y_test = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

    # We reserve the last 10000 training examples for validation.
    #X_train, X_val = X_train[:-10000], X_train[-10000:]
    #y_train, y_val = y_train[:-10000], y_train[-10000:]
    
    # We just return all the arrays in order, as expected in main().
    # (It doesn't matter how we do this as long as we can read them again.)
    return X_train, y_train, X_test, y_test

In [4]:
X_train, y_train, X_test, y_test = load_dataset()

In [5]:
y_train


array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [9]:
X_train.shape

(60000L, 1L, 28L, 28L)

## Setup Neural Network

In [16]:
nnet = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden1', layers.DenseLayer),
        ('hidden2', layers.DenseLayer),
        ('hidden3', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=( None, 1, 28, 28),  #96x96 input pixels per batch
    hidden1_num_units=500,  # number of units in hidden layer
    hidden2_num_units=500,  # number of units in hidden layer
    hidden3_num_units=500,  # number of units in hidden layer
    output_nonlinearity= softmax,  # output layer uses identity function
    output_num_units=10,  # 30 target values

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,

    regression=False,  # flag to indicate we're dealing with regression problem
    max_epochs=15,  # we want to train this many epochs
    verbose=1,
    )

## Fit Model

In [17]:
nnet.fit(X_train, y_train)

# Neural Network with 898510 learnable parameters

## Layer information

  #  name     size
---  -------  -------
  0  input    1x28x28
  1  hidden1  500
  2  hidden2  500
  3  hidden3  500
  4  output   10

  epoch    trn loss    val loss    trn/val    valid acc  dur
-------  ----------  ----------  ---------  -----------  ------
      1     0.48612     0.24273    2.00270      0.92736  29.35s
      2     0.19434     0.16889    1.15073      0.94902  29.93s
      3     0.13634     0.14031    0.97168      0.95918  29.90s
      4     0.10304     0.12255    0.84082      0.96318  29.30s
      5     0.08023     0.11040    0.72676      0.96610  30.75s
      6     0.06319     0.10234    0.61740      0.96826  30.85s
      7     0.04995     0.09663    0.51692      0.97035  29.03s
      8     0.03955     0.09340    0.42342      0.97176  28.68s
      9     0.03137     0.09049    0.34670      0.97301  28.91s
     10     0.02482     0.08397    0.29563      0.97626  28.57s
     11     0.01952     0.0

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x0000000009BE5400>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x0000000009A58588>,
     check_input=True, custom_scores=None, hidden1_num_units=500,
     hidden2_num_units=500, hidden3_num_units=500,
     input_shape=(None, 1, 28, 28),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden1', <class 'lasagne.layers.dense.DenseLayer'>), ('hidden2', <class 'lasagne.layers.dense.DenseLayer'>), ('hidden3', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=15, more_params={},
     objective=<function objective at 0x0000000009BDD518>,
     objective_loss_function=<function categorical_crossentropy at 0x0000000009734978>,
     on_batch_finished=[],
     on_epoch_finished=[<nolearn.lasagne.handlers.PrintLog instance at 0x000000000E632048>],
     on_training_finished=[

## Predict digits for Test Data

In [19]:
y_pred = nnet.predict(X_test)

## Calculate Test Accuracy

In [20]:
s=0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        s += 1

print ("Accuracy: %f", s*100.0/len(y_pred))

('Accuracy: %f', 97.64)


## Display Confusion Matrix

In [21]:
from sklearn.metrics import confusion_matrix

In [22]:
tbl = confusion_matrix(y_test, y_pred, labels=[0,1,2,3,4,5,6,7,8,9])

In [23]:
print tbl

[[ 974    0    1    0    0    1    1    1    2    0]
 [   0 1127    3    1    0    1    2    0    1    0]
 [   5    1 1015    2    2    0    3    3    1    0]
 [   0    0    8  987    0    4    0    2    1    8]
 [   1    0    8    0  952    1    1    0    0   19]
 [   3    0    0    9    1  876    2    0    1    0]
 [   5    3    0    0    6   11  932    0    1    0]
 [   1    8   15    3    1    0    1  982    4   13]
 [   4    0    6   14    1    5    3    3  935    3]
 [   2    2    0    7    6    2    2    3    1  984]]


## Setup Neural Network with Regularization

In [25]:
nnet = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden1', layers.DenseLayer),
        ('dropout1', layers.DropoutLayer),
        ('hidden2', layers.DenseLayer),
        ('dropout2', layers.DropoutLayer),
        ('hidden3', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=( None, 1, 28, 28),  # input shape
    hidden1_num_units=500,  # number of units in hidden layer
    hidden2_num_units=500,  # number of units in hidden layer
    hidden3_num_units=500,  # number of units in hidden layer
    dropout1_p = 0.2,
    dropout2_p = 0.1,
    output_nonlinearity= softmax,  # output layer uses identity function
    output_num_units=10,  # 10 target values

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,

    regression=False,  # flag to indicate we're dealing with regression problem
    max_epochs=15,  # we want to train this many epochs
    verbose=1,
    )

In [26]:
nnet.fit(X_train, y_train)

# Neural Network with 898510 learnable parameters

## Layer information

  #  name      size
---  --------  -------
  0  input     1x28x28
  1  hidden1   500
  2  dropout1  500
  3  hidden2   500
  4  dropout2  500
  5  hidden3   500
  6  output    10

  epoch    trn loss    val loss    trn/val    valid acc  dur
-------  ----------  ----------  ---------  -----------  ------
      1     0.53153     0.23203    2.29076      0.93086  33.37s
      2     0.21592     0.16158    1.33633      0.95019  32.22s
      3     0.15651     0.12677    1.23459      0.96235  34.07s
      4     0.12442     0.10812    1.15075      0.96735  31.85s
      5     0.10266     0.09806    1.04690      0.97018  32.54s
      6     0.08510     0.08618    0.98746      0.97343  32.80s
      7     0.07189     0.08144    0.88271      0.97509  34.00s
      8     0.06303     0.07856    0.80232      0.97709  34.49s
      9     0.05417     0.07940    0.68232      0.97584  36.52s
     10     0.04713     0.07755    0.60768    

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x0000000009BE5400>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x0000000009A58588>,
     check_input=True, custom_scores=None, dropout1_p=0.2, dropout2_p=0.1,
     hidden1_num_units=500, hidden2_num_units=500, hidden3_num_units=500,
     input_shape=(None, 1, 28, 28),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden1', <class 'lasagne.layers.dense.DenseLayer'>), ('dropout1', <class 'lasagne.layers.noise.DropoutLayer'>), ('hidden2', <class 'lasagne.layers.dense.DenseLayer'>), ('dropout2', <class 'lasagne.layers.noise.DropoutLayer'>), ('hidden3', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=15, more_params={},
     objective=<function objective at 0x0000000009BDD518>,
     objective_loss_function=<function categorical_crossentropy at 0x0000000009734978